In [ ]:
!pip install paho-mqtt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/99.4 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for paho-mqtt: filename=paho_mqtt-1.6.1-py3-none-any.whl size=62118 sha256=1b830481c6232c169c3f98501525a7111102044a2f267d19efc8b6ded31e1484
  Stored in directory: /root/.cache/pip/wheels/8b/bb/0c/79444d1dee20324d442856979b5b519b48828b0bd3d05df84a
Successfully built paho-mqtt


In [ ]:
import os
import re
import pickle

from sklearn import linear_model
import paho.mqtt.client as mqtt
import pandas

# 把資料存進 CSV 檔案

In [ ]:
mqtt_server = "MQTT Server"
mqtt_topics = "MQTT Topics"
data_file = "./data.csv"

file = open(data_file, "wt+", encoding="utf-8-sig")
print("Soil Humid,DHT Humid,Temperature", file=file)

# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))

    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    client.subscribe(mqtt_topics)

# The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, msg):
    soil_humid = re.findall(r"Soil Humid: \d+", msg)[0][12:]
    dht_humid = re.findall(r"DHT Humid: \d+\.\d+", msg)[0][11:]
    temp = re.findall(r"Temp: \d+\.\d+", msg)[0][6:]
    print("{},{},{}".format(soil_humid, dht_humid, temp), file=file)


client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect(mqtt_server, 1883, 60)

# Blocking call that processes network traffic, dispatches callbacks and
# handles reconnecting.
# Other loop*() functions are available that give a threaded interface and a
# manual interface.
client.loop_forever()
file.close()

# 線性回歸

In [ ]:
data_file = "./data.csv"

data = pandas.read_csv(data_file)
soil = data["Soil Humid"].to_list()
learn = [[i[1], i[2]] for i in data[["DHT Humid", "DHT Humid"]].itertuples()]

reg = linear_model.LinearRegression()
reg.fit(learn, soil)

with open("./model.bin", "wb+") as f:
  f.write(pickle.dumps(reg))